In [29]:
from leagues.models import Competition,Season
from base_app.helpers import best_fuzzy_match
from games.models import Game
from datetime import datetime
import pandas as pd
import numpy as np
import os

In [32]:
EV_PATH = "D:\MetaData"
ST_PATH = "D:\MetaData-shot"

In [62]:
event_files = []
for base,_,files in os.walk(EV_PATH):
    for f in files:
        if 'shot' not in f :
            event_files.append(f"{base}\{f}")

shot_files = []
for base,_,files in os.walk(ST_PATH):
    for f in files:
        shot_files.append(f"{base}\{f}")

event_files = list(set(event_files))
shot_files = list(set(shot_files))
len(event_files),len(shot_files)

(603, 337)

In [57]:
(603, 332)

(603, 332)

In [66]:
map_example = []
non_mapped_season = []

for season in Season.objects.all():
    conf = season.competition.confederation
    region = season.competition.country
    
    e_name = season.competition.name_scoresaway.replace(' ','_').replace(".","")
    s_name = season.competition.name_fotmob.replace(' ','_').replace(".","")
    
    e_s_name = season.name.replace('/','_').replace(' ','_')
    s_s_name = season.name_fotmob.replace('/','_').replace(' - ','_')

    if season.competition_id == 27 :
        e_search_str = f"D:\\MetaData\\{conf}\\{region}\\{e_name}_{e_s_name}_events.xlsx"
        s_search_str = [f"D:\\MetaData-shot\\{conf}\\{region}\\{s_name}_{e_s_name}_Apertura_shots.xlsx",
                        f"D:\\MetaData-shot\\{conf}\\{region}\\{s_name}_{e_s_name}_Clausura_shots.xlsx"]
        if e_search_str in event_files and s_search_str[0] in shot_files and s_search_str[1] in shot_files  :
            map_example.append((season,e_search_str,"__||__".join(s_search_str)))
    else:
        e_search_str = f"D:\\MetaData\\{conf}\\{region}\\{e_name}_{e_s_name}_events.xlsx"
        s_search_str = f"D:\\MetaData-shot\\{conf}\\{region}\\{s_name}_{s_s_name}_shots.xlsx"

        if e_search_str in event_files and s_search_str in shot_files  :
            map_example.append((season,e_search_str,s_search_str))
        else:
            if season.season_shot_url and e_search_str not in event_files :
                # print(f"Event = {e_search_str in event_files} | Shot = {s_search_str in shot_files} |\t{season}")
                print(f"{season.competition.confederation},{season.competition.country},{season.competition.competition_name},{season.name},{is_season_active(season)}")
            non_mapped_season.append((season,e_search_str))
    

UEFA,Republic of Ireland,Ireland Premier Division,2025,False
AFC,Japan,J1 League,2026/2027,True
CONMEBOL,Brazil,Brasileiro Serie A,2026,True
CONCACAF,USA,Major League Soccer,2026,True
UEFA,Norway,Eliteserien,2026,True
UEFA,Republic of Ireland,Ireland Premier Division,2026,True
UEFA,Sweden,Allsvenskan,2026,True
AFC,India,Indian Super League,2025/2026,True


In [65]:
from datetime import datetime

def is_season_active(season: Season) -> bool:
    current_date = datetime.now().date()
    comp = season.competition

    start = comp.season_start
    end = comp.season_end

    if start < end:
        expected_year = str(current_date.year)
    else:
        if current_date.month >= start:
            expected_year = str(current_date.year)
        else:
            expected_year = str(current_date.year - 1)

    return season.name.lower().startswith(expected_year)


In [22]:
e = []
s = []
for x in non_mapped_season :
    comp = x[0].competition
    if comp.shot_data_url :
        s.append(x)
    else:
        e.append(comp.competition_name)
    # o = {}
    # p = pd.read_excel(x[1])
    # o['Comp'] = x[0].competition.competition_name
    # o['cnt'] = p.shape[0]
    # t.append(o)

In [23]:
data = []
for season,event_path in s:
    t = {}
    t['confederation'] = season.competition.confederation
    t['region'] = season.competition.country
    t['competition'] = season.competition.competition_name
    t['season'] = season.name_fotmob
    conf = season.competition.confederation
    region = season.competition.country
    
    e_name = season.competition.name_scoresaway.replace(' ','_').replace(".","")
    s_name = season.competition.name_fotmob.replace(' ','_').replace(".","")
    
    e_s_name = season.name.replace('/','_').replace(' ','_')
    s_s_name = season.name_fotmob.replace('/','_').replace(' - ','_')

    if season.competition_id == 27 :
        e_search_str = f"D:\\MetaData\\{conf}\\{region}\\{e_name}_{e_s_name}_events.xlsx"
        s_search_str = [f"D:\\MetaData-shot\\{conf}\\{region}\\{s_name}_{e_s_name}_Apertura_shots.xlsx",
                        f"D:\\MetaData-shot\\{conf}\\{region}\\{s_name}_{e_s_name}_Clausura_shots.xlsx"]
        if e_search_str in event_files and s_search_str[0] in shot_files and s_search_str[1] in shot_files  :
            map_example.append((season,e_search_str,"__||__".join(s_search_str)))

        t['comp_e'] = season.competition.name_scoresaway
        t['comp_s'] = season.competition.name_fotmob
        t['event_path'] = e_search_str
        t['event_exist'] = e_search_str in event_files
    
        t['shot_path'] = s_search_str
        t['shot_exist'] = True if sum(1 for x in s_search_str if x in shot_files)== 2 else False
    else:
        e_search_str = f"D:\\MetaData\\{conf}\\{region}\\{e_name}_{e_s_name}_events.xlsx"
        s_search_str = f"D:\\MetaData-shot\\{conf}\\{region}\\{s_name}_{s_s_name}_shots.xlsx"
        t['comp_e'] = season.competition.name_scoresaway
        t['comp_s'] = season.competition.name_fotmob
        t['event_path'] = e_search_str
        t['event_exist'] = e_search_str in event_files
    
        t['shot_path'] = s_search_str
        t['shot_exist'] = s_search_str in shot_files
    data.append(t)
    

In [15]:
pd.DataFrame(data).to_excel('CheckFax2.xlsx')

In [48]:
## SHOT EVENT

In [ ]:
https://www.scoresway.com/en_GB/soccer/premier-division-2025/3zoeegue4xtul58c8ksahcjdg/results
https://www.scoresway.com/en_GB/soccer/premier-division-2025/3zoeegue4xtul58c8ksahcjdg/results